# Prep 1: steps to install the packages

In [1]:
!pip install py_entitymatching
!pip install scipy
!pip install numpy
!pip install pandas

     |████████████████████████████████| 2.0MB 2.9MB/s 
     |████████████████████████████████| 788kB 39.9MB/s 
     |████████████████████████████████| 655kB 17.4MB/s 
  Stored in directory: /root/.cache/pip/wheels/8a/11/32/31986854281ff89654b9e67c1cb5d94dd8f87d7408aaef3c91
  Stored in directory: /root/.cache/pip/wheels/69/a2/f7/b748da307ea9120633a58847b0ecec23c97cfccdf81d875282
  Stored in directory: /root/.cache/pip/wheels/60/7a/90/37ae79d15afb482c2fa1a58a45fadd1945ed1d09fc8d50d4b6
Successfully built py-entitymatching py-stringsimjoin py-stringmatching


In [12]:
from google.colab import files
files.upload()
files.upload()
files.upload()
files.upload()
files.upload()

Saving tablea to tablea


Saving tableb to tableb


Saving candidate to candidate


Saving prediction to prediction


Saving 400 to 400


{'400': b'A_id,B_id,label\n402,1389,1\n2917,11,0\n802,2749,0\n3324,3834,0\n4407,3834,0\n2118,3846,0\n4406,2745,0\n1010,1290,1\n1881,612,1\n878,464,1\n1745,2745,0\n703,2745,0\n4370,3627,0\n1019,1068,1\n3842,3922,0\n677,608,0\n1159,3181,0\n4568,3795,0\n1470,2745,0\n1156,3383,0\n2202,715,0\n603,2411,0\n319,3383,0\n2292,1937,0\n4485,3181,0\n4243,2100,0\n3547,3790,0\n2335,3834,0\n1964,2839,1\n1261,2898,1\n1398,1810,1\n3742,3383,0\n1870,11,0\n3414,42,0\n1215,3395,1\n4521,3301,1\n1131,822,0\n4040,1744,0\n914,1295,0\n3560,3834,0\n435,651,0\n2962,3203,0\n1793,2665,1\n2644,3261,1\n2707,3383,0\n1581,3846,0\n3979,3203,0\n866,3645,0\n865,1260,1\n935,55,1\n1928,1954,0\n3736,3627,0\n2144,984,0\n105,425,0\n1394,249,0\n246,3834,0\n316,2745,0\n1282,3627,0\n4201,502,0\n3582,3627,0\n3147,1954,0\n1650,1148,0\n3418,2197,0\n4585,3690,0\n2368,3685,0\n3398,3181,0\n2302,3834,0\n812,2753,0\n4641,3275,1\n1972,11,0\n4066,832,0\n3118,2188,0\n1654,2745,0\n2139,11,0\n1173,3834,0\n3964,3834,0\n2173,11,0\n3650,3809,1\n

# Prep 2: enter the file location on your harddisk

In [0]:
table_a = 'tablea'
table_b = 'tableb'
candidate_set = 'candidate'
prediction_set = 'prediction'

# Prep 3: reading the files into pandas dataframe

In [0]:
import pandas as pd

In [0]:
dfa = pd.read_csv(table_a)
dfb = pd.read_csv(table_b)
dfc = pd.read_csv(candidate_set)
dfp = pd.read_csv(prediction_set)

# Module: debug_blocker
# Description: debug the blocking rule using the below script to ensure you are not dropping true matches
# Note: You need to run Prep 1 and 2 in order to run this

In [0]:
# Example input format:
# Format of table_a:
# _id, attribute1, attribute2, ....., attributen

# Format of table_b:
# _id, attribute1, attribute2, ....., attributen

# Format of candidate_set
# A_id,B_id
# where A_id is _id from table_a and B_id is the _id column value from table_b

In [0]:
import py_entitymatching as em

def run_debug_blocker(table_a, table_b, table_a_key, table_b_key, candidate_set):
    dfl = em.read_csv_metadata(table_a, key=table_a_key)
    dfr = em.read_csv_metadata(table_b, key=table_b_key)

    # reading the candidate set and adding key
    dfcand = pd.read_csv(candidate_set)
    dfcand.drop_duplicates(inplace=True)
    dfcand.to_csv('cand_set_with_index.csv', index_label='id')

    dfcset = em.read_csv_metadata('cand_set_with_index.csv', key='id', ltable=dfl, 
                                  rtable=dfr, fk_ltable='A_id', fk_rtable='B_id')

    # running debug blocker to identify the records in A x B \ C
    debug_file = em.debug_blocker(dfcset, dfl, dfr)
    
    return debug_file

In [28]:
debug_file = run_debug_blocker(table_a, table_b, '_id', '_id', candidate_set)

Metadata file is not present in the given path; proceeding to read the csv file.
Metadata file is not present in the given path; proceeding to read the csv file.
Metadata file is not present in the given path; proceeding to read the csv file.


# Module: estimate_precision_recall
# Description: the below code helps you get an estimation of P/R on the candidate set.

In [0]:
import pandas as pd
from scipy.stats import norm
from numpy import sqrt


delta = .05
Z = norm.ppf(1 - (delta / 2))

def estimate_PR(labeled_pairs, reduced_cands, predicted_matches):
    '''
    labeled_pairs - a pandas dataframe with schema id1,id2,label
                    Note label needs to be Boolean

    reduced_cands - a pandas dataframe with schema id1,id2
    predicted_matches - a pandas dataframe with schema id1,id2
    
    return:
        ( (recall lower bound, recall upper bound), (precision lower bound, precision upper bound) )
    '''

    reduced_cand_set = set(zip(reduced_cands.id1, reduced_cands.id2))
    predicted_matches = set(zip(predicted_matches.id1, predicted_matches.id2))
    
    # estimate the recall
    # number of positives in the labeled sample
    actual_pos = float(labeled_pairs.label.sum())
    # the maximum number of postives in the candidate set
    max_actual_pos = float(actual_pos + len(reduced_cand_set) - len(labeled_pairs))
    
    # true positives in the labeled sample
    true_pos = float(labeled_pairs.apply(lambda x : (x['id1'], x['id2']) in predicted_matches and x['label'], axis=1).sum())
    #estimated recall
    recall = float(true_pos / actual_pos)

    recall_error = Z * sqrt( int(((recall * (1 - recall)) / actual_pos) * ((max_actual_pos - actual_pos) / max_actual_pos - 1) ))


    # estimate Precision
    labeled_set  = set(zip(labeled_pairs.id1, labeled_pairs.id2))
    predicted_pos = float(len(labeled_set & predicted_matches))
    
    predicted_pos_in_reduced_cand_set = float(len(reduced_cand_set & predicted_matches))
    
    alpha =  predicted_pos_in_reduced_cand_set / len(predicted_matches)
    precision = alpha * (true_pos / predicted_pos)
    
    precision_error = alpha * Z * sqrt( ((precision * (1 - precision)) / predicted_pos) * (float((len(predicted_matches) - predicted_pos)) / (len(predicted_matches)  - 1)) )
    

    return ((recall - recall_error, recall + recall_error),
            (precision - precision_error, 1 if precision+precision_error else precision+precision_error))

In [78]:
labeled = pd.read_csv('400')
labeled = labeled.rename(columns={'A_id':'id1','B_id':'id2','label':'label'})
dfc = dfc.rename(columns={'A_id':'id1','B_id':'id2'})
dfp = dfp.rename(columns={'A_id':'id1','B_id':'id2','matching_probability':'matching_probability'})
print(estimate_PR(labeled, dfc, dfp))

((1.0, 1.0), (0.9489684697118062, 1))
